In [1]:
%load_ext autoreload
%autoreload 


In [2]:
import timeit
import copy
import concurrent.futures 
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import batoid

from ghosts import plotter
from ghosts import simulator
from ghosts import tweak_optics
from ghosts import beam
from ghosts import geom


from ghosts.analysis import compute_ghost_separations, reduce_ghosts
from ghosts.analysis import match_ghosts, compute_reduced_distance, compute_2d_reduced_distance, find_nearest_ghost
from ghosts.analysis import make_data_frame

from ghosts.beam_configs import BEAM_CONFIG_0, BEAM_CONFIG_1, BEAM_CONFIG_2, FAST_BEAM_CONFIG_1



In [3]:
from ghosts.constants import *
print(f'LSST Camera has {LSST_CAMERA_PIXEL_DENSITY_MM2:.0f} pixels per mm^2')

LSST Camera has 36000 pixels per mm^2


In [4]:
# CCOB like geometry, i.e. lenses + filters
telescope = batoid.Optic.fromYaml("../data/LSST_CCOB_r.yaml")

In [5]:
# test list of optics
assert tweak_optics.get_list_of_optics(telescope)==['L1', 'L2', 'Filter', 'L3', 'Detector'], 'Not a CCOB optical setup'

# Make refractive interfaces partially reflective
# Call on current telescope
tweak_optics.make_optics_reflective(telescope)


0.02

In [6]:
# Run simulation with standard beam config
traceFull, rForward, rReverse, rays = simulator.run_simulation(telescope, beam_config=BEAM_CONFIG_1)


In [7]:
# Analyze
spots_data, _spots = reduce_ghosts(rForward)
data_frame = make_data_frame(spots_data, BEAM_CONFIG_1)


In [8]:
data_frame.head()

,config,n_photons,beam_x,beam_y,beam_width,beam_theta,beam_phi,index,name,pos_x,std_x,pos_y,std_y,width_x,width_y,radius,radius_err,surface,pixel_signal
0,1,1000,0.1,0,0.00125,0.0,0.0,0,"(L1_exit, L1_entrance)",0.044591,0.000282,1.069581e-05,0.000282,0.001096,0.001088,0.001092,0.000004,3.745156,529.217844
1,1,1000,0.1,0,0.00125,0.0,0.0,1,"(L2_exit, L1_entrance)",0.142048,0.000897,3.407246e-05,0.000897,0.003487,0.003465,0.003476,0.000011,37.958472,48.161494
2,1,1000,0.1,0,0.00125,0.0,0.0,2,"(L2_exit, L1_exit)",0.218113,0.001384,5.231793e-05,0.001384,0.005383,0.005321,0.005352,0.000031,89.980509,21.154759
3,1,1000,0.1,0,0.00125,0.0,0.0,3,"(L2_entrance, L1_entrance)",0.003179,0.000018,7.625250e-07,0.000018,0.000072,0.000078,0.000075,0.000003,0.017475,108928.791000
4,1,1000,0.1,0,0.00125,0.0,0.0,4,"(L2_entrance, L1_exit)",0.055178,0.000349,1.323544e-05,0.000349,0.001356,0.001346,0.001351,0.000005,5.733517,345.687169


In [9]:
# Test building several beams
x_scan = [0., 0.05, 0.1, 0.15, 0.2]
t_scan = [0, 5, 10, 15]

In [10]:
beam_scan_x = beam.build_translation_set(BEAM_CONFIG_0, 'x', x_scan, base_id=10000)
beam_scan_t = beam.build_rotation_set(BEAM_CONFIG_1, 'theta', t_scan, base_id=20000)

In [11]:
beam_scan_x_panda = beam.concat_dicts(beam_scan_x)
beam_scan_x_panda.head()

,beam_id,wl,n_photons,radius,x_offset,y_offset,z_offset,theta,phi
10000,10000,5.000000e-07,1000,0.00125,0.00,0,0,0.0,0.0
10001,10001,5.000000e-07,1000,0.00125,0.05,0,0,0.0,0.0
10002,10002,5.000000e-07,1000,0.00125,0.10,0,0,0.0,0.0
10003,10003,5.000000e-07,1000,0.00125,0.15,0,0,0.0,0.0
10004,10004,5.000000e-07,1000,0.00125,0.20,0,0,0.0,0.0


In [12]:
beam_scan_t_panda = beam.concat_dicts(beam_scan_t)
beam_scan_t_panda.head()

,beam_id,wl,n_photons,radius,x_offset,y_offset,z_offset,theta,phi
20000,20000,5.000000e-07,1000,0.00125,0.1,0,0,0,0.0
20001,20001,5.000000e-07,1000,0.00125,0.1,0,0,5,0.0
20002,20002,5.000000e-07,1000,0.00125,0.1,0,0,10,0.0
20003,20003,5.000000e-07,1000,0.00125,0.1,0,0,15,0.0


In [14]:
# test make fast beams on the data frame for all beam configurations
beam_scan_t_panda['n_photons']=10
beam_scan_t_panda.head()

,beam_id,wl,n_photons,radius,x_offset,y_offset,z_offset,theta,phi
20000,20000,5.000000e-07,10,0.00125,0.1,0,0,0,0.0
20001,20001,5.000000e-07,10,0.00125,0.1,0,0,5,0.0
20002,20002,5.000000e-07,10,0.00125,0.1,0,0,10,0.0
20003,20003,5.000000e-07,10,0.00125,0.1,0,0,15,0.0
